In [144]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [145]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse

In [222]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *

In [180]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'],header=0)
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
test_users_with_city = users_df[users_df.in_test==1][['user_id','city']]
print(len(test_users),len(users_df),len(test_users_with_city))

16967 1252801 16967


In [184]:
reviews, user_agg, org_agg = load_reviews(users_df=users_df, orgs_df=orgs_df)
orgs_df = orgs_df.merge(org_agg, on='org_id')
users_df = users_df.merge(user_agg, on='user_id', how='left')

In [181]:
test_users_df = test_users_with_city\
    .merge(reviews[reviews.rating>=4.0][['user_id','org_id']], on='user_id', how='left')\
    .groupby(['user_id','city'])\
    .aggregate(list)\
    .reset_index()

In [313]:
train_reviews, test_reviews = train_test_split(reviews, users_df)

In [315]:
result, encs = prepare_reviews_i2i(train_reviews, users_df, orgs_df)

In [316]:
m = reviews_matrix(result, encs)
cc_mat = m.T * m

In [317]:
J_sim = CC_2_J(cc_mat)
pmi_sim = CC_2_pmi(cc_mat)
ease_sim = ease_solution(cc_mat, l2=30000).T

In [ ]:
J_preds = i2i_predict(J_sim, test_reviews, encs)
pmi_preds = i2i_predict(pmi_sim, test_reviews, encs)
ease_preds = i2i_predict(ease_sim, test_reviews, encs)

In [ ]:
merged = i2i_predict_merged([J_sim, ease_sim],[1,1.3], test_users_df, encs)

In [ ]:
recall()

In [237]:
validate_preds(fallback_with_top_recs(ease_preds, orgs_df), orgs_df, users_df)

All good


In [241]:
save_predictions(fallback_with_top_recs(merged, orgs_df), path='merged_ans.csv')

In [219]:
save_predictions(fallback_with_top_recs(pmi_preds, orgs_df), path='pmi_ans.csv')

In [248]:
save_predictions(fallback_with_top_recs(ease_preds, orgs_df), path='ease_ans.csv')

In [196]:
save_predictions(fallback_with_top_recs(J_preds,orgs_df), path='J_ans.csv')

In [23]:
aspects = load_aspects()
rubrics = load_rubrics(orgs_df=orgs_df)
features = load_features(orgs_df=orgs_df)